In [178]:
import mat73
import numpy as np


In [39]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV


filepath = "SSTindivsomata_GLM.mat"

data_dict = mat73.loadmat(filepath)

num_animals = len(data_dict['animal']['ShiftLrate'])

reorganized_data = {}

for animal_idx in range(num_animals):
    neuron_list = []
    
    num_neurons = data_dict['animal']['ShiftR'][animal_idx].shape[2]
    
    for neuron_idx in range(1,num_neurons):
        neuron_data = []
        neuron_data.append(data_dict['animal']['ShiftLrate'][animal_idx])               # Lick rate
        neuron_data.append(data_dict['animal']['ShiftV'][animal_idx])                   # Reward location (valve opening)
        neuron_data.append(data_dict['animal']['ShiftRunning'][animal_idx])             # Running speed  

        # TODO: Add 5 spatial features
        # TODO: add combo of lick+valve

        
        neuron_data.append(data_dict['animal']['ShiftR'][animal_idx][:, :, neuron_idx]) # Neuron activity
        combined_matrix = np.stack(neuron_data, axis=1) 
        neuron_list.append(combined_matrix)

    reorganized_data[f'animal_{animal_idx + 1}'] = neuron_list

print(f"Shape of the combined matrix for the first neuron of animal_1: {reorganized_data['animal_1'][0].shape}")
print(f"Total number of animals in dictionary: {len(reorganized_data)}")
print(f"Number of neurons for animal_1: {len(reorganized_data['animal_1'])}")

Shape of the combined matrix for the first neuron of animal_1: (50, 4, 213)
Total number of animals in dictionary: 10
Number of neurons for animal_1: 5


In [151]:
GLM_params = {}

for animal in reorganized_data:
    GLM_params[animal] = {}
    for i,neuron in enumerate(reorganized_data[animal]):
        avg_neuron_data = np.nanmean(neuron[:,:,1:], axis=2)
        avg_neuron_data = (avg_neuron_data - np.mean(avg_neuron_data, axis=0)) / np.std(avg_neuron_data, axis=0) # Z-score the data
        design_matrix_X = avg_neuron_data[:, :3]
        neuron_activity = avg_neuron_data[:, 3]
        
        ridge_cv = RidgeCV(alphas=[0.1, 1, 10, 100, 1000, 5000], store_cv_results=True)
        ridge_cv.fit(design_matrix_X, neuron_activity)

        GLM_params[animal][i] = {'weights': ridge_cv.coef_, 
                                 'intercept': ridge_cv.intercept_, 
                                 'alpha': ridge_cv.alpha_, 
                                 'cv_results': np.mean(ridge_cv.cv_results_, axis=0)}
        